In [1]:
import os
import pandas as pd
import openpyxl as xl
import json

In [2]:
def list_all_files(base_path):
    all_files = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            all_files.append(os.path.join(root, file))
    return all_files

In [3]:
path = '/Users/oomrawat/Desktop/04_FSP/FINC308 - Investment Analysis/Group Project/Industry Wise Data'

all_files_list = list_all_files(path)
all_files_w_year = []

for file in all_files_list:
    if file[-8:] != 'DS_Store':
        year_month = os.path.dirname(file)[-7:]
        if year_month[0] == '/':
            year_month = year_month[1:]
        all_files_w_year.append((file,year_month))

files_to_consider = []
for file in all_files_w_year:
    year_month = file[1]
    file_path = file[0]
    if file_path[-15:] == '_optimized.xlsx':
        files_to_consider.append((file_path, year_month))
    
len(files_to_consider)

19

In [4]:
stocks_selected_no_zero_weightage = {}
stocks_selected_top_2_overall_highest_weightage = {}
intersection_of_both = {}

In [5]:
years = [('2023','12')]

In [6]:
for year,month in years:
    year_month = year+'_'+month
    stocks_selected_no_zero_weightage[year_month] = {}
    stocks_selected_top_2_overall_highest_weightage[year_month] = {}
    intersection_of_both[year_month] = {}

In [7]:
for file_path, year in files_to_consider:
    
    industry = os.path.basename(file_path)[:-15]

    wb = xl.load_workbook(file_path)

    ws = wb['max_sharpe_ratio']
    symbols = [cell.value for cell in ws['A'] if cell.value is not None][1:]

    max_shr = [cell.value[1:] for cell in ws['B'] if cell.value is not None][1:]
    max_shr = [float(val) for val in max_shr]

    ws = wb['min_variance']
    min_var = [cell.value[1:] for cell in ws['B'] if cell.value is not None][1:]
    min_var = [float(val) for val in min_var]

    ws = wb['max_returns']
    max_ret = [cell.value[1:] for cell in ws['B'] if cell.value is not None][1:]
    max_ret = [float(val) for val in max_ret]

    df = pd.DataFrame(columns=['Symbols', 'Weights_Shr', 'Weights_Var', 'Weights_Ret'])
    df['Symbols'] = symbols
    df['Weights_Shr'] = max_shr
    df['Weights_Var'] = min_var
    df['Weights_Ret'] = max_ret

    df['Total Weight'] = df[['Weights_Shr', 'Weights_Var', 'Weights_Ret']].sum(axis=1)

    df_sorted = df.sort_values(by='Total Weight', ascending=False)

    stocks_selected_top_2_overall_highest_weightage[year][industry] = list(df_sorted['Symbols'].head(1))

    df_nonzero = df[(df['Weights_Shr'] != 0) & (df['Weights_Var'] != 0) & (df['Weights_Ret'] != 0)]

    df_nonzero = df_nonzero.sort_values(by='Total Weight', ascending=False)

    stocks_selected_no_zero_weightage[year][industry] = list(df_nonzero['Symbols'].head(1))

In [8]:
industry_data = pd.read_csv('ind_niftytotalmarket_list.csv')
industries_list = list(industry_data['Industry'].unique())
industries_list.remove('Diversified')
industries_list.remove('Utilities')
industries_list.remove('Forest Materials')

industries_list, len(industries_list)

(['Financial Services',
  'Capital Goods',
  'Construction Materials',
  'Healthcare',
  'Chemicals',
  'Information Technology',
  'Power',
  'Metals & Mining',
  'Services',
  'Oil Gas & Consumable Fuels',
  'Fast Moving Consumer Goods',
  'Consumer Services',
  'Construction',
  'Textiles',
  'Automobile and Auto Components',
  'Consumer Durables',
  'Realty',
  'Telecommunication',
  'Media Entertainment & Publication'],
 19)

In [9]:
for year, month in years:
    year_month = year+'_'+month
    for industry in industries_list:
        print(year_month,industry)
        try:
            list1 = stocks_selected_no_zero_weightage[year_month][industry]
            list2 = stocks_selected_top_2_overall_highest_weightage[year_month][industry]
            intersection_set = set(list1).intersection(list2)
            intersection_list = list(intersection_set)
            intersection_of_both[year_month][industry] = intersection_list
        except:
            intersection_of_both[year_month][industry] = []

2023_12 Financial Services
2023_12 Capital Goods
2023_12 Construction Materials
2023_12 Healthcare
2023_12 Chemicals
2023_12 Information Technology
2023_12 Power
2023_12 Metals & Mining
2023_12 Services
2023_12 Oil Gas & Consumable Fuels
2023_12 Fast Moving Consumer Goods
2023_12 Consumer Services
2023_12 Construction
2023_12 Textiles
2023_12 Automobile and Auto Components
2023_12 Consumer Durables
2023_12 Realty
2023_12 Telecommunication
2023_12 Media Entertainment & Publication


In [10]:
# sum_no_zero_weightage = 0  
for k1,v1 in stocks_selected_no_zero_weightage.items():
    sum_no_zero_weightage = 0  
    for k2,v2 in v1.items():
        sum_no_zero_weightage += len(v2)
    print(sum_no_zero_weightage)

print('#############')

# sum_2_overall_highest_weightage = 0  
for k1,v1 in stocks_selected_top_2_overall_highest_weightage.items():
    sum_2_overall_highest_weightage = 0  
    for k2,v2 in v1.items():
        sum_2_overall_highest_weightage += len(v2)
    print(sum_2_overall_highest_weightage)

print('#############')

# sum_intersection_of_both = 0  
for k1,v1 in intersection_of_both.items():
    sum_intersection_of_both = 0  
    for k2,v2 in v1.items():
        sum_intersection_of_both += len(v2)
    print(sum_intersection_of_both)

# sum_no_zero_weightage, sum_2_overall_highest_weightage, intersection_of_both

19
#############
19
#############
19


In [11]:
json_file_path = 'industryWise_monthWise_stocks_no_zero_weightage.json'

with open(json_file_path, 'w') as json_file:
    json.dump(stocks_selected_no_zero_weightage, json_file)

json_file_path = 'industryWise_monthWise_stocks_overall_highest_weightage.json'

with open(json_file_path, 'w') as json_file:
    json.dump(stocks_selected_top_2_overall_highest_weightage, json_file)

json_file_path = 'industryWise_monthWise_stocks_intersection_of_both.json'

with open(json_file_path, 'w') as json_file:
    json.dump(intersection_of_both, json_file)